In [2]:
import os
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

In [3]:
def file_name(file_dir):   
    L=[]   
    for root, dirs, files in os.walk(file_dir):  
        for file in files:  
            if os.path.splitext(file)[1] == '.h5':
                L.append(os.path.join(root, file)[29:-3])  
    return L # file name = id

In [4]:
def open_genre_1(lst):
    f = open('genre.txt','r')
    dic = {}
    Set = []
    id_genre = {}
    for i in f.readlines():
        content = i.split()
        content[-1] = content[-1][:-1]
        if len(content) == 3:
            content[1] = content[1]+" "+content[2]
        if len(content) == 1:
            continue
        dic[content[0]] = content[1]
        Set.append(content[1])
    genre = list(set(Set))
    # print(genre)
    # print(len(set(Set)))
    number = [0]*15
    for j in lst:
        if j in dic:
            id_genre[j] = dic[j]
    for m in id_genre:
        number[genre.index(id_genre[m])] +=1
    # print(number)
    # print(sum(number))
    return id_genre, genre

In [5]:
def open_genre_2(lst,id_genre):
    g = open('newgenre.txt','r')
    dic2 = {}
    # Set2 = []
    for k in g.readlines():
        content = k.split()
        if len(content) == 3:
            content[1] = content[1]+" "+content[2]
        if len(content) == 1:
            continue
        dic2[content[0]] = content[1]
    #     Set2.append(content[1])
    # genre2 = list(set(Set2))
    # print(genre2)
    # print(len(set(Set2)))
    for item in lst:
        if (item not in id_genre) and (item in dic2) and dic2[item] != "Pop_Rock" and dic2[item] != "International" and dic2[item] != "Vocal":
            id_genre[item] = dic2[item]
    # print(len(id_genre))
    return id_genre

In [6]:
def data_pre_process(updated_id_genre):
    processed_data = {}
    x = []
    y = []
    lenlist = []
    for k,v in updated_id_genre.items():
        lst = []
        for root, dirs, files in os.walk("MillionSongSubset/data"):
            for file in files:
                if os.path.splitext(file)[0] == k:
                    f = h5py.File(os.path.join(root, file), 'r')
                    for i in f['analysis']["segments_timbre"]:
                        for m in list(i):
                            lst.append(m)
        if len(lst)>=4800:
            Array = np.array(lst[:4800])
            processed_data[k] = [v,Array]
            x.append(Array)
            y.append(v)
    return processed_data, x, y

In [7]:
lst = file_name("MillionSongSubset/data")

id_genre, genre = open_genre_1(lst)
# genre = ['Rock', 'Country', 'New Age', 'Jazz', 'Folk', 'Blues', 'Pop', 'Metal', 'World', 'Reggae', 'Electronic', 'Punk', 'Rap', 'RnB', 'Latin']
updated_id_genre = open_genre_2(lst,id_genre)
print(len(updated_id_genre))

3396


In [8]:
processed_data, x, y= data_pre_process(updated_id_genre)
print(len(processed_data))
print(len(x))

3190
3190


In [9]:
X_all = np.array(x).reshape(3190,12,20,20)
y_number = [genre.index(y[i]) for i in range(len(y))]
y_all = np.array(y_number)
X_train = X_all[:1400]
y_train = y_all[:1400]
X_val = X_all[1400:1590]
y_val = y_all[1400:1590]
X_test = X_all[1590:]
y_test = y_all[1590:]
# X_train = X_all[:2000]
# y_train = y_all[:2000]
# X_val = X_all[2000:2190]
# y_val = y_all[2000:2190]
# X_test = X_all[2190:]
# y_test = y_all[2190:]

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(12, 32, 3, 1, 1) 
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.fc1 = nn.Linear(64 * 20 * 20, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 15)
        #Three linear layers

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x  = x.view(-1,64 * 20 * 20)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
# init network
conv_net = ConvNet()
print('model structure: ', conv_net)
# init optimizer
optimizer = optim.Adam(conv_net.parameters(), lr=0.001)
# set loss function
criterion = nn.CrossEntropyLoss()

# prepare for mini-batch stochastic gradient descent
n_iteration = 20
batch_size = 32
n_data = X_train.shape[0]
n_batch = int(np.ceil(n_data/batch_size))

# convert X_train and X_val to tensor and flatten them
X_train_tensor = torch.Tensor(X_train)
X_val_tensor = torch.Tensor(X_val)

# convert training label to tensor and to type long
y_train_tensor = torch.Tensor(y_train).long()
y_val_tensor = torch.Tensor(y_val).long()

print('X train tensor shape:', X_train_tensor.shape)

model structure:  ConvNet(
  (conv1): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=25600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=15, bias=True)
)
X train tensor shape: torch.Size([1400, 12, 20, 20])


In [12]:
def get_correct_and_accuracy(y_pred, y):
    # y_pred is the nxC prediction scores
    # give the number of correct and the accuracy
    n = y.shape[0]
    # find the prediction class label
    _ ,pred_class = y_pred.max(dim=1)
    correct = (pred_class == y).sum().item()
    return correct ,correct/n

In [13]:
train_loss_list = np.zeros(n_iteration)
train_accu_list = np.zeros(n_iteration)
val_loss_list = np.zeros(n_iteration)
val_accu_list = np.zeros(n_iteration)
for i in range(n_iteration):
    # first get a minibatch of data
    running_loss = 0
    correct_train = 0
    for j in range(n_batch):
        batch_start_index = j*batch_size
        # get data batch from the normalized data
        X_batch = X_train_tensor[batch_start_index:batch_start_index+batch_size]
        # get ground truth label y
        y_batch = y_train_tensor[batch_start_index:batch_start_index+batch_size]
        pred = conv_net(X_batch)
        loss = criterion(pred, y_batch)
        running_loss += loss.item()
        correct_train += get_correct_and_accuracy(pred, y_batch)[0]
        # update 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    ave_train_loss = running_loss / n_batch
    pred_val = conv_net(X_val_tensor)
    loss_val = criterion(pred_val, y_val_tensor)
    val_loss = loss_val
    train_accu = correct_train/float(n_data)
    val_accu =  get_correct_and_accuracy(pred_val, y_val_tensor)[1]

    print("Iter %d ,Train loss: %.3f, Train acc: %.3f, Val loss: %.3f, Val acc: %.3f" 
          %(i ,ave_train_loss, train_accu, val_loss, val_accu)) 
    ## add to the logs so that we can use them later for plotting
    train_loss_list[i] = ave_train_loss
    train_accu_list[i] = train_accu
    val_loss_list[i] = val_loss
    val_accu_list[i] = val_accu

Iter 0 ,Train loss: 3.240, Train acc: 0.361, Val loss: 2.291, Val acc: 0.316
Iter 1 ,Train loss: 1.918, Train acc: 0.417, Val loss: 2.031, Val acc: 0.389
Iter 2 ,Train loss: 1.327, Train acc: 0.587, Val loss: 2.039, Val acc: 0.395
Iter 3 ,Train loss: 0.541, Train acc: 0.859, Val loss: 2.285, Val acc: 0.379
Iter 4 ,Train loss: 0.190, Train acc: 0.947, Val loss: 2.460, Val acc: 0.353
Iter 5 ,Train loss: 0.161, Train acc: 0.952, Val loss: 3.283, Val acc: 0.258
Iter 6 ,Train loss: 0.124, Train acc: 0.970, Val loss: 2.690, Val acc: 0.411
Iter 7 ,Train loss: 0.081, Train acc: 0.985, Val loss: 4.277, Val acc: 0.395
Iter 8 ,Train loss: 0.045, Train acc: 0.991, Val loss: 3.276, Val acc: 0.416
Iter 9 ,Train loss: 0.008, Train acc: 0.998, Val loss: 3.573, Val acc: 0.432
Iter 10 ,Train loss: 0.012, Train acc: 0.998, Val loss: 3.729, Val acc: 0.432
Iter 11 ,Train loss: 0.001, Train acc: 1.000, Val loss: 4.071, Val acc: 0.395
Iter 12 ,Train loss: 0.001, Train acc: 1.000, Val loss: 4.019, Val acc: 0.

In [14]:
X_test_tensor = torch.Tensor(X_test)
y_test_tensor = torch.Tensor(y_test).long()
pred = conv_net(X_test_tensor)
correct_train = get_correct_and_accuracy(pred, y_test_tensor)[1]
print("Test accuracy: ", correct_train)

Test accuracy:  0.25625


In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1000)
X_pca = X_all.reshape(3190,4800)
pca.fit(X_pca)
Data = X_pca
print(Data.shape)
X_train_pca=Data[:1500]
X_val_pca=Data[1500:1690]
X_test_pca=Data[1690:]

(3190, 4800)


In [16]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train_pca, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [1500, 1400]

In [ ]:
correct = 0
for i in range(len(X_val_pca)):
    if clf.predict(X_val_pca[i].reshape(1,-1))==y_val[i]:
        correct +=1
correct_rate = correct/len(y_val)
print(correct_rate)

In [ ]:
correct = 0
print(len(X_test_pca))
for i in range(len(X_test_pca)):
    a = clf.predict(X_test_pca[i].reshape(1,-1))
    if a==y_test[i]:
        correct +=1
correct_rate = correct/len(y_test)
print(correct_rate)